In [1]:
import pandas as pd
import sqlite3

In [2]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("my_database.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [51]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

cervezas
bares


In [55]:
query = '''
CREATE TABLE reparto (
	CodE VARCHAR(2) NOT NULL,
	CodB VARCHAR(3) NOT NULL,
	CodC VARCHAR(2) NOT NULL,
	Fecha DATE NOT NULL,
	Cantidad SMALLINT,
	PRIMARY KEY (CodE,CodB,CodC,fecha)
);
'''
crsr.execute(query)

In [10]:
query = '''
DROP TABLE cervezas 
'''
crsr.execute(query)

In [31]:
query = '''
INSERT INTO cervezas VALUES ('01', 'Botella', 0.2, 3600)
'''
crsr.execute(query)

In [63]:
query = '''
SELECT * FROM reparto 
'''
sql_query(query)

,CodE,CodB,CodC,Fecha,Cantidad
0,1,001,01,2005-10-21,240
1,1,001,02,2005-10-21,48
2,1,002,03,2005-10-22,60
3,1,004,05,2005-10-22,4
4,2,002,03,2005-10-22,48
5,2,002,05,2005-10-23,2
6,2,004,01,2005-10-23,480
7,2,004,02,2005-10-24,72
8,3,003,03,2005-10-24,48
9,3,003,04,2005-10-25,20


In [59]:
query = '''
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) 
VALUES  ('1','001','01','2005-10-21',240 ),
( '1','001','02','2005-10-21',48 ),
( '1','002','03','2005-10-22',60 ),
( '1','004','05','2005-10-22',4 ),
( '2','002','03','2005-10-22',48 ),
( '2','002','05','2005-10-23',2 ),
( '2','004','01','2005-10-23',480 ),
( '2','004','02','2005-10-24',72 ),
( '3','003','03','2005-10-24',48 ),
( '3','003','04','2005-10-25',20 )
;
'''
crsr.execute(query)

# INSERT INTO cervezas VALUES ('02','Botella',0.33,1200 )
# INSERT INTO cervezas VALUES ('03', 'Lata', 0.33, 2400)
# INSERT INTO cervezas VALUES ('04', 'Botella', 1, 288)
# INSERT INTO cervezas VALUES ('05', 'Barril', 60, 30)

In [64]:
connection.commit()
connection.close()

In [73]:
#1
query = '''
SELECT DISTINCT
empleados.Nombre
FROM empleados 
INNER JOIN reparto
ON empleados.CodE = reparto.CodE
INNER JOIN bares
ON reparto.CodB = bares.CodB
WHERE bares.nombre = 'Stop' and reparto.Fecha BETWEEN '17/10/2005' and '23/10/2005'
'''
sql_query(query)

,Nombre
0,Prudencio Caminero


In [86]:
#2
query = '''
SELECT DISTINCT
bares.Cif,
bares.nombre
FROM bares
INNER JOIN reparto 
ON bares.CodB = reparto.CodB 
INNER JOIN cervezas
ON reparto.CodC = cervezas.CodC
WHERE cervezas.envase = 'Botella' and cervezas.Capacidad < 1
ORDER BY bares.Localidad
'''
sql_query(query)

,Cif,Nombre
0,44444444Z,Otra Ronda
1,11111111X,Stop


In [109]:
#3
query = '''
SELECT
bares.nombre,
cervezas.envase,
cervezas.capacidad,
reparto.Fecha,
reparto.Cantidad
FROM reparto
INNER JOIN empleados
ON reparto.CodE = empleados.CodE
INNER JOIN bares
ON reparto.CodB = bares.CodB
INNER JOIN cervezas
ON cervezas.CodC = reparto.CodC
WHERE empleados.Nombre = 'Prudencio Caminero'
'''
sql_query(query)

,Nombre,Envase,Capacidad,Fecha,Cantidad
0,Stop,Botella,0.20,2005-10-21,240
1,Stop,Botella,0.33,2005-10-21,48
2,Las Vegas,Lata,0.33,2005-10-22,60
3,Otra Ronda,Barril,60.00,2005-10-22,4


In [18]:
#9
query = '''
SELECT 
CodE
FROM reparto
GROUP BY CodE
ORDER BY COUNT( DISTINCT Fecha) DESC
LIMIT 1
'''
sql_query(query)

,CodE
0,2


In [ ]:
#9
query = '''
UPDATE empleados
SET sueldo = sueldo * 1.05
WHERE CodE = 2
'''
crsr.execute(query)

In [11]:
#9
query = '''
SELECT * FROM empleados
'''
sql_query(query)

,CodE,Nombre,Sueldo
0,1,Prudencio Caminero,120000
1,2,Vicente Merario,115500
2,3,Valentin Siempre,100000


In [19]:
#9
query = '''
UPDATE empleados
SET sueldo = sueldo * 1.05
WHERE CodE IN (
SELECT 
CodE
FROM reparto
GROUP BY CodE
ORDER BY COUNT( DISTINCT Fecha) DESC LIMIT 1)
'''
crsr.execute(query)

In [20]:
#9
query = '''
SELECT * FROM empleados
'''
sql_query(query)

,CodE,Nombre,Sueldo
0,1,Prudencio Caminero,120000
1,2,Vicente Merario,121275
2,3,Valentin Siempre,100000


In [34]:
#9
query = '''
SELECT 
CodE,
COUNT( DISTINCT Fecha) as dias_trabajados,
COUNT(CodE) as n_reparto
FROM reparto
GROUP BY CodE
ORDER BY 2 DESC,3 DESC
'''
sql_query(query)

,CodE,dias_trabajados,n_reparto
0,2,3,4
1,1,2,4
2,3,2,2


In [38]:
#9
query = '''
CREATE VIEW dias_empelados as
SELECT 
CodE,
COUNT( DISTINCT Fecha) as dias_trabajados
FROM reparto
GROUP BY CodE
ORDER BY 2 DESC
'''
crsr.execute(query)

In [47]:
#9
query = '''
SELECT 
MIN(dias_trabajados) as min_dias
FROM dias_empelados
'''
sql_query(query)

,min_dias
0,2


In [48]:
#9
query = '''
SELECT 
* FROM dias_empelados
'''
sql_query(query)

,CodE,dias_trabajados
0,2,3
1,1,2
2,3,2


In [44]:
#9
query = '''
SELECT a.CodE
FROM dias_empelados a
INNER JOIN (SELECT 
MIN(dias_trabajados) as min_dias
FROM dias_empelados) b
ON a.dias_trabajados = b.min_dias
'''
sql_query(query)

,CodE
0,1
1,3


In [ ]:
#9
query = '''
UPDATE empleados
SET sueldo = sueldo * 1.05
WHERE CodE = 1 and CodE = 3
'''
crsr.execute(query)

In [ ]:
#9
query = '''
UPDATE empleados
SET sueldo = sueldo * 1.1
WHERE CodE = 2
'''
crsr.execute(query)

In [60]:
query = '''
INSERT INTO reparto VALUES ('5', '002', '03', '2005-10-20',270)
'''
crsr.execute(query)

In [61]:

query = '''
SELECT  * FROM reparto
'''
sql_query(query)

,CodE,CodB,CodC,Fecha,Cantidad
0,1,001,01,2005-10-21,240
1,1,001,02,2005-10-21,48
2,1,002,03,2005-10-22,60
3,1,004,05,2005-10-22,4
4,2,002,03,2005-10-22,48
5,2,002,05,2005-10-23,2
6,2,004,01,2005-10-23,480
7,2,004,02,2005-10-24,72
8,3,003,03,2005-10-24,48
9,3,003,04,2005-10-25,20


In [75]:
# INNER JOIN
query = '''
SELECT a.*,
SUM(b.Cantidad) as cantidades
FROM empleados a
INNER JOIN reparto b
ON a.CodE=b.CodE
GROUP BY 1,2,3
'''
sql_query(query)

,CodE,Nombre,Sueldo,cantidades
0,1,Prudencio Caminero,120000,352
1,2,Vicente Merario,121275,602
2,3,Valentin Siempre,100000,68


In [76]:
# LEFT JOIN
query = '''
SELECT a.*,
SUM(b.Cantidad) as cantidades
FROM empleados a
LEFT JOIN reparto b
ON a.CodE=b.CodE
GROUP BY 1,2,3
'''
sql_query(query)

,CodE,Nombre,Sueldo,cantidades
0,1,Prudencio Caminero,120000,352.0
1,2,Vicente Merario,121275,602.0
2,3,Valentin Siempre,100000,68.0
3,4,Ambrosio Pérez,150000,NaN


In [77]:
#RIGHT JOIN
query = '''
SELECT a.*,
SUM(b.Cantidad) as cantidades
FROM reparto b
LEFT JOIN empleados a
ON a.CodE=b.CodE
GROUP BY 1,2,3
'''
sql_query(query)

,CodE,Nombre,Sueldo,cantidades
0,None,None,NaN,270
1,1,Prudencio Caminero,120000.0,352
2,2,Vicente Merario,121275.0,602
3,3,Valentin Siempre,100000.0,68


In [78]:
# FULL JOIN = LEFT JOIN + RIGHT JOIN

query = '''
SELECT a.*,
SUM(b.Cantidad) as cantidades
FROM empleados a
LEFT JOIN reparto b
ON a.CodE=b.CodE
GROUP BY 1,2,3
UNION
SELECT a.*,
SUM(b.Cantidad) as cantidades
FROM reparto b
LEFT JOIN empleados a
ON a.CodE=b.CodE
GROUP BY 1,2,3
'''
sql_query(query)

,CodE,Nombre,Sueldo,cantidades
0,None,None,NaN,270.0
1,1,Prudencio Caminero,120000.0,352.0
2,2,Vicente Merario,121275.0,602.0
3,3,Valentin Siempre,100000.0,68.0
4,4,Ambrosio Pérez,150000.0,NaN
